# Color Quantization

exploring images with OpenCV (Open Source Computer Vision) library

__challenge:__ working with images from a RESTful JSON API</br>
<a href="https://www.pexels.com">Photos provided by Pexels</a>

#### references: 
https://www.pexels.com/api/documentation</br>
https://github.com/AguilarLagunasArturo/pexels-api</br>
https://docs.opencv.org/3.4/d1/d5c/tutorial_py_kmeans_opencv.html</br>

## First get some images

For the purposes of comparing image colors, you'll search a free image repository for photos matching two different kinds of music, pop music and punk! 

Purely for research purposes, this decision is based on a hypothesis that punk music photos would contain more dark (and neon?) colors, and of course who doesn't have a soft spot for the pinks or happy colors associated with so called pop music?! I will admit that popular is a pretty vague term, but you are just experimenting.

In [1]:
# # install Pexels API v1 with Python once only, with pip because it's not hosted on anaconda, and quietly in the notebook
# !pip install pexels-api --quiet

In [2]:
# optionally, check that your environment variable(s) are active with the env
!conda env config vars list

MY_KEY = kPu1pIMFXQdQun0Q9bqhzV3aKidiWZk40VjyBk0obCBeaHQVSOtvxGrn
CLOUD_KEY = 635838363772573
CLOUD_SECRET = b0okpKNp9T3WGZSC1q3eFO0rD4M


In [3]:
import os  # standard Python module for operating system-based tasks like working with file paths
import cv2  # OpenCV computer vision library used for data clustering, i.e., measuring similarities within color spaces
import numpy as np  # for working with images as numerical arrays
import matplotlib.pyplot as plt  # used to plot or display image data in cartesian 2D space
from urllib.request import urlopen  # for opening URLs


In [4]:
from pexels_api import API  # import API class from pexels_api package
# load your Pexels API key from a conda variable stored with the activated conda environment
PEXELS_API_KEY = os.environ["MY_KEY"]
# Create API object
api = API(PEXELS_API_KEY)

In [5]:
# search five 'punk' photos
api.search('punk', page=1, results_per_page=5)
# get photo entries (a list of photo objects)
photos_punk = api.get_entries()
# loop the five photos
for photo in photos_punk:
  # print photographer
  print('Photographer: ', photo.photographer)
  # print url
  print('Photo url: ', photo.url)
  # print original size url
  print('Photo original size: ', photo.original)

Photographer:  William Matt
Photo url:  https://www.pexels.com/photo/funky-skull-graffiti-on-locked-roll-down-black-door-953457/
Photo original size:  https://images.pexels.com/photos/953457/pexels-photo-953457.jpeg
Photographer:  Clem Onojeghuo
Photo url:  https://www.pexels.com/photo/man-sitting-on-guitar-amplifier-playing-electric-guitar-375893/
Photo original size:  https://images.pexels.com/photos/375893/pexels-photo-375893.jpeg
Photographer:  Darya Sannikova
Photo url:  https://www.pexels.com/photo/photo-of-woman-sitting-on-stairs-1989836/
Photo original size:  https://images.pexels.com/photos/1989836/pexels-photo-1989836.jpeg
Photographer:  Cheef
Photo url:  https://www.pexels.com/photo/man-covering-his-mouth-2080736/
Photo original size:  https://images.pexels.com/photos/2080736/pexels-photo-2080736.jpeg
Photographer:  Jaime Rivera
Photo url:  https://www.pexels.com/photo/woman-in-black-leather-jacket-3598252/
Photo original size:  https://images.pexels.com/photos/3598252/pexel

In [6]:
# search five 'pop' photos; explicitly include 'music' to avoid bubbles and other images that 'pop'
api.search('pop music', page=1, results_per_page=5)
# get photo entries (a list of photo objects)
photos_pop = api.get_entries()
# loop the five photos
for photo in photos_pop:
  # print photographer
  print('Photographer: ', photo.photographer)
  # print url
  print('Photo url: ', photo.url)
  # print original size url
  print('Photo original size: ', photo.original)

Photographer:  Jacob Morch
Photo url:  https://www.pexels.com/photo/people-inside-dark-room-with-spotlights-426976/
Photo original size:  https://images.pexels.com/photos/426976/pexels-photo-426976.jpeg
Photographer:  mali maeder
Photo url:  https://www.pexels.com/photo/man-playing-pioneer-dj-turntable-219101/
Photo original size:  https://images.pexels.com/photos/219101/pexels-photo-219101.jpeg
Photographer:  Edward Eyer
Photo url:  https://www.pexels.com/photo/man-playing-guitar-811838/
Photo original size:  https://images.pexels.com/photos/811838/pexels-photo-811838.jpeg
Photographer:  Hannah Gibbs
Photo url:  https://www.pexels.com/photo/photo-of-man-holding-microphone-949274/
Photo original size:  https://images.pexels.com/photos/949274/pexels-photo-949274.jpeg
Photographer:  Lucas Allmann
Photo url:  https://www.pexels.com/photo/people-performing-on-stage-442540/
Photo original size:  https://images.pexels.com/photos/442540/pexels-photo-442540.jpeg


#### Skip these next few cells if you want; warning just practice ahead

In [7]:
# optionally, peek the array of photo objects returned from Pexels API (just to inspect what's happening)
photos_punk  # or peek at photos_pop


In [8]:
# optionally, print its class; never a bad idea to be sure you know what you're working with
print(type(photos_punk))

<class 'list'>


In [9]:
# just to dissect this more, use enumerate() to iterate over the list and show its index and objects explicitly
for index , item in enumerate(photos_punk):
    print(index,item)

0 <pexels_api.tools.photo.Photo object at 0x7fe620270fa0>
1 <pexels_api.tools.photo.Photo object at 0x7fe620271a80>
2 <pexels_api.tools.photo.Photo object at 0x7fe620270f40>
3 <pexels_api.tools.photo.Photo object at 0x7fe620271a50>
4 <pexels_api.tools.photo.Photo object at 0x7fe6202711e0>


In [10]:
# list comprehension for no reason but practicing python and remembering how lists behave
print([list((i, photos_punk[i])) for i in range(len(photos_punk))])

[[0, <pexels_api.tools.photo.Photo object at 0x7fe620270fa0>], [1, <pexels_api.tools.photo.Photo object at 0x7fe620271a80>], [2, <pexels_api.tools.photo.Photo object at 0x7fe620270f40>], [3, <pexels_api.tools.photo.Photo object at 0x7fe620271a50>], [4, <pexels_api.tools.photo.Photo object at 0x7fe6202711e0>]]


## Okay, now download the images

So photo objects are not so useful until we can _see_ them! 

If you try to read Pexel images from URL directly to an image using urllib, you will get a 403 forbidden error. I'm guessing that Pexels has forbidden hotlinking directly to its images (although it's not stated in their terms), as this can be a heavy draw on its server. Else, I'm stuck for some other reason.</br>

So, downloading is one way to go from here. (I'd love to hear about alternatives!)

In [11]:
# commented out as returning HTTPError: HTTP Error 403: Forbidden
# # make a function to read an image from its url
# def url_to_image(url, readFlag=cv2.IMREAD_COLOR):
#     # download the image, convert it to a NumPy array, and then
#     # read it into OpenCV format (i.e., decode it)
#     resp = urlopen(url)  # open a connection to the url
#     image = np.asarray(bytearray(resp.read()), dtype="uint8")  # convert the raw byte-sequence to a one-dimensional NumPy array, i.e., a long list of pixels
#     image = cv2.imdecode(image, readFlag)  # reshape the array into a 2D format, assuming 3 components per pixel (Red, Green, and Blue components)

#     # return the image
#     return image

# # loop the five photos
# for photo in photos_punk:
#     # download the image from its original size URL and display it
# #     print("downloading %s" % (photo.original))
# #     image = url_to_image(photo.original)  # returns HTTPError: HTTP Error 403: Forbidden
#     print("downloading %s" % (photo.url))
#     image = url_to_image(photo.url)
#     cv2.imshow("Image", image)
#     cv2.waitKey(0)

### Enter Cloudinary or your favorite media hosting service

I recommend downloading cloudinary with pip as I checked and conda did have a version but it was behind pip's so better to be up to date to capture any bug and security fixes.
https://pypi.org/project/cloudinary/<br>

You can visit [cloudinary.com](https://cloudinary.com/) to get started for free and get code chunks in python and many other languages also.

In [12]:
# install cloudinary just once, quiet in the notebook
!pip install cloudinary --quiet

In [15]:
# get conda environment variables
CLOUDINARY_API_KEY = os.environ["CLOUD_KEY"]
CLOUDINARY_API_SECREY = os.environ["CLOUD_SECRET"]

# Import cloudinary
from cloudinary.uploader import upload
from cloudinary.utils import cloudinary_url
import cloudinary  # added to resolve NameError: name 'cloudinary' is not defined on cloudinary.config()

# Config
cloudinary.config(
  cloud_name = "dtartuanr",  # if you copy code directly from cloudinary.com this should appear as your username (or replace it manually)
  api_key = CLOUDINARY_API_KEY,
  api_secret = CLOUDINARY_API_SECREY,
  secure = True
)

# Upload a test
upload("https://images.pexels.com/photos/426976/pexels-photo-426976.jpeg", public_id="people-inside-dark-room-with-spotlights-426976")

# Transform
url, options = cloudinary_url("people-inside-dark-room-with-spotlights-426976", width=100, height=150, crop="fill")
asset  # returning NameError: name 'asset' is not defined


NameError: name 'asset' is not defined

## Scratch area for reference code

In [ ]:
# # read image as a numpy array from file path
# img_arr = img.imread('/content/sample_image.jpg')
# print(img_arr.shape)